# Correction to Wind_dir differences
### According to this logic in Tableau script language
IF (((SUM([KLO Wind dir])-LOOKUP(SUM([KLO Wind dir]),-1)))>=(-180)
AND ((SUM([KLO Wind dir])-LOOKUP(SUM([KLO Wind dir]),-1))<=180))
THEN (SUM([KLO Wind dir])-LOOKUP(SUM([KLO Wind dir]),-1))
ELSEIF ((SUM([KLO Wind dir])-LOOKUP(SUM([KLO Wind dir]),-1))<(-180))
THEN (360+(SUM([KLO Wind dir])-LOOKUP(SUM([KLO Wind dir]),-1)))
ELSEIF((SUM([KLO Wind dir])-LOOKUP(SUM([KLO Wind dir]),-1))>180)
THEN ((SUM([KLO Wind dir])-LOOKUP(SUM([KLO Wind dir]),-1))-360)
END

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
# Python libs
import numpy as np
import pandas as pd
from os.path import isfile, join
import os.path
import re
from google.colab import drive
import sys
import datetime as datetime
import xgboost as xgb
import pickle

from sklearn.model_selection import RandomizedSearchCV

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import recall_score, precision_score, f1_score
import time
import datetime as dt

import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV

from imblearn.over_sampling import SMOTE




/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [0]:
data_path = 'gdrive/My Drive/bise_data'

In [0]:
pd.set_option('display.max_columns', None)

In [4]:
#drive.mount('/gdrive')
drive.mount('/gdrive',force_remount=True)

Mounted at /gdrive


In [5]:
os.chdir('/')
os.getcwd()

'/'

In [0]:
# custom scripts
sys.path.append('gdrive/My Drive/Colab Notebooks')
import data_handler as dh
import feature_handler as fh
import data_processor as dp
import constants as const
import work_utils as wu
import plot_handler as ph
import xgb_handler as xh

In [27]:
dh.master_dataset()

'model_data_all_features_extended.csv.zip'

In [29]:
file_master = join(data_path, dh.master_dataset())
df_master = pd.read_csv(file_master, index_col=0)
df_master.head()

,datetime,KLO_Air_temp,KLO_Hum,KLO_Hum_delta_3,KLO_Hum_delta_6,KLO_QNH,KLO_Wind_dir,KLO_Wind_dir_delta,KLO_Wind_dir_delta-3,KLO_Wind_dir_delta-6,KLO_Wind_dir_spread30,KLO_Wind_dir_spread60,KLO_Wind_max_EW,KLO_Wind_max_NS,KLO_Wind_max,KLO_Wind_max_delta_3,KLO_Wind_max_delta_6,KLO_Wind_speed_EW,KLO_Wind_speed_NS,KLO_Wind_speed,KLO_Wind_speed_delta_3,KLO_Wind_speed_delta_6,BIZ_Air_temp,BIZ_Hum,BIZ_Hum_delta_3,BIZ_Hum_delta_6,BIZ_QNH,BIZ_Wind_dir,BIZ_Wind_dir_delta,BIZ_Wind_dir_delta-3,BIZ_Wind_dir_delta-6,BIZ_Wind_dir_spread30,BIZ_Wind_dir_spread60,BIZ_Wind_max_EW,BIZ_Wind_max_NS,BIZ_Wind_max,BIZ_Wind_max_delta_3,BIZ_Wind_max_delta_6,BIZ_Wind_speed_EW,BIZ_Wind_speed_NS,BIZ_Wind_speed,BIZ_Wind_speed_delta_3,BIZ_Wind_speed_delta_6,BUS_Air_temp,BUS_Hum,BUS_Hum_delta_3,BUS_Hum_delta_6,BUS_QNH,BUS_Wind_dir,BUS_Wind_dir_delta,BUS_Wind_dir_delta-3,BUS_Wind_dir_delta-6,BUS_Wind_dir_spread30,BUS_Wind_dir_spread60,BUS_Wind_max_EW,BUS_Wind_max_NS,BUS_Wind_max,BUS_Wind_max_delta_3,BUS_Wind_max_delta_6,BUS_Wind_speed_EW,BUS_Wind_speed_NS,BUS_Wind_speed,BUS_Wind_speed_delta_3,BUS_Wind_speed_delta_6,GUT_Air_temp,GUT_Hum,GUT_Hum_delta_3,GUT_Hum_delta_6,GUT_QNH,GUT_Wind_dir,GUT_Wind_dir_delta,GUT_Wind_dir_delta-3,GUT_Wind_dir_delta-6,GUT_Wind_dir_spread30,GUT_Wind_dir_spread60,GUT_Wind_max_EW,GUT_Wind_max_NS,GUT_Wind_max,GUT_Wind_max_delta_3,GUT_Wind_max_delta_6,GUT_Wind_speed_EW,GUT_Wind_speed_NS,GUT_Wind_speed,GUT_Wind_speed_delta_3,GUT_Wind_speed_delta_6,KOP_Air_temp,KOP_Hum,KOP_Hum_delta_3,KOP_Hum_delta_6,KOP_QNH,KOP_Wind_dir,KOP_Wind_dir_delta,KOP_Wind_dir_delta-3,KOP_Wind_dir_delta-6,KOP_Wind_dir_spread30,KOP_Wind_dir_spread60,KOP_Wind_max_EW,KOP_Wind_max_NS,KOP_Wind_max,KOP_Wind_max_delta_3,KOP_Wind_max_delta_6,KOP_Wind_speed_EW,KOP_Wind_speed_NS,KOP_Wind_speed,KOP_Wind_speed_delta_3,KOP_Wind_speed_delta_6,MOA_Air_temp,MOA_Hum,MOA_Hum_delta_3,MOA_Hum_delta_6,MOA_QNH,MOA_Wind_dir,MOA_Wind_dir_delta,MOA_Wind_dir_delta-3,MOA_Wind_dir_delta-6,MOA_Wind_dir_spread30,MOA_Wind_dir_spread60,MOA_Wind_max_EW,MOA_Wind_max_NS,MOA_Wind_max,MOA_Wind_max_delta_3,MOA_Wind_max_delta_6,MOA_Wind_speed_EW,MOA_Wind_speed_NS,MOA_Wind_speed,MOA_Wind_speed_delta_3,MOA_Wind_speed_delta_6,MUB_Air_temp,MUB_Hum,MUB_Hum_delta_3,MUB_Hum_delta_6,MUB_QNH,MUB_Wind_dir,MUB_Wind_dir_delta,MUB_Wind_dir_delta-3,MUB_Wind_dir_delta-6,MUB_Wind_dir_spread30,MUB_Wind_dir_spread60,MUB_Wind_max_EW,MUB_Wind_max_NS,MUB_Wind_max,MUB_Wind_max_delta_3,MUB_Wind_max_delta_6,MUB_Wind_speed_EW,MUB_Wind_speed_NS,MUB_Wind_speed,MUB_Wind_speed_delta_3,MUB_Wind_speed_delta_6,PAY_Air_temp,PAY_Hum,PAY_Hum_delta_3,PAY_Hum_delta_6,PAY_QNH,PAY_Wind_dir,PAY_Wind_dir_delta,PAY_Wind_dir_delta-3,PAY_Wind_dir_delta-6,PAY_Wind_dir_spread30,PAY_Wind_dir_spread60,PAY_Wind_max_EW,PAY_Wind_max_NS,PAY_Wind_max,PAY_Wind_max_delta_3,PAY_Wind_max_delta_6,PAY_Wind_speed_EW,PAY_Wind_speed_NS,PAY_Wind_speed,PAY_Wind_speed_delta_3,PAY_Wind_speed_delta_6,PUY_Air_temp,PUY_Hum,PUY_Hum_delta_3,PUY_Hum_delta_6,PUY_QNH,PUY_Wind_dir,PUY_Wind_dir_delta,PUY_Wind_dir_delta-3,PUY_Wind_dir_delta-6,PUY_Wind_dir_spread30,PUY_Wind_dir_spread60,PUY_Wind_max_EW,PUY_Wind_max_NS,PUY_Wind_max,PUY_Wind_max_delta_3,PUY_Wind_max_delta_6,PUY_Wind_speed_EW,PUY_Wind_speed_NS,PUY_Wind_speed,PUY_Wind_speed_delta_3,PUY_Wind_speed_delta_6,REH_Air_temp,REH_Hum,REH_Hum_delta_3,REH_Hum_delta_6,REH_QNH,REH_Wind_dir,REH_Wind_dir_delta,REH_Wind_dir_delta-3,REH_Wind_dir_delta-6,REH_Wind_dir_spread30,REH_Wind_dir_spread60,REH_Wind_max_EW,REH_Wind_max_NS,REH_Wind_max,REH_Wind_max_delta_3,REH_Wind_max_delta_6,REH_Wind_speed_EW,REH_Wind_speed_NS,REH_Wind_speed,REH_Wind_speed_delta_3,REH_Wind_speed_delta_6,SHA_Air_temp,SHA_Hum,SHA_Hum_delta_3,SHA_Hum_delta_6,SHA_QNH,SHA_Wind_dir,SHA_Wind_dir_delta,SHA_Wind_dir_delta-3,SHA_Wind_dir_delta-6,SHA_Wind_dir_spread30,SHA_Wind_dir_spread60,SHA_Wind_max_EW,SHA_Wind_max_NS,SHA_Wind_max,SHA_Wind_max_delta_3,SHA_Wind_max_delta_6,SHA_Wind_speed_EW,SHA_Wind_speed_NS,SHA_Wind_speed,SHA_Wind_speed_delta_3,SHA_Wind_speed_delta

### Create a smaller dataset to work on. Copy the new values back to original dataset

In [33]:
cols_wind_dir = [c for c in df_master.columns if re.search('Wind_dir$', c)]
df_wind_dir = df_master[cols_wind_dir].copy()
df_wind_dir.head()

,KLO_Wind_dir,BIZ_Wind_dir,BUS_Wind_dir,GUT_Wind_dir,KOP_Wind_dir,MOA_Wind_dir,MUB_Wind_dir,PAY_Wind_dir,PUY_Wind_dir,REH_Wind_dir,SHA_Wind_dir,SMA_Wind_dir,TAE_Wind_dir,VAD_Wind_dir,WAE_Wind_dir,WYN_Wind_dir
0,138.0,283.0,303.0,207.0,191.0,146.0,94.0,262.0,9.0,104.0,233.0,201.0,68.0,129.0,152.0,206.0
1,149.0,265.0,302.0,200.0,154.0,177.0,98.0,236.0,338.0,93.0,231.0,202.0,358.0,129.0,143.0,205.0
2,145.0,238.0,261.0,198.0,222.0,165.0,99.0,171.0,356.0,69.0,229.0,193.0,344.0,124.0,127.0,213.0
3,127.0,271.0,292.0,236.0,222.0,151.0,94.0,190.0,295.0,79.0,230.0,200.0,229.0,141.0,143.0,237.0
4,131.0,270.0,294.0,184.0,240.0,150.0,88.0,88.0,135.0,98.0,265.0,216.0,227.0,153.0,142.0,233.0


In [0]:
df_wind_dir

In [34]:
for c in cols_wind_dir:
  new_name = c + '_delta'
  index = df_wind_dir.columns.get_loc(c) + 1
  df_wind_dir.insert(index, new_name, df_wind_dir[c])
 
df_wind_dir.head()

,KLO_Wind_dir,KLO_Wind_dir_delta,BIZ_Wind_dir,BIZ_Wind_dir_delta,BUS_Wind_dir,BUS_Wind_dir_delta,GUT_Wind_dir,GUT_Wind_dir_delta,KOP_Wind_dir,KOP_Wind_dir_delta,MOA_Wind_dir,MOA_Wind_dir_delta,MUB_Wind_dir,MUB_Wind_dir_delta,PAY_Wind_dir,PAY_Wind_dir_delta,PUY_Wind_dir,PUY_Wind_dir_delta,REH_Wind_dir,REH_Wind_dir_delta,SHA_Wind_dir,SHA_Wind_dir_delta,SMA_Wind_dir,SMA_Wind_dir_delta,TAE_Wind_dir,TAE_Wind_dir_delta,VAD_Wind_dir,VAD_Wind_dir_delta,WAE_Wind_dir,WAE_Wind_dir_delta,WYN_Wind_dir,WYN_Wind_dir_delta
0,138.0,138.0,283.0,283.0,303.0,303.0,207.0,207.0,191.0,191.0,146.0,146.0,94.0,94.0,262.0,262.0,9.0,9.0,104.0,104.0,233.0,233.0,201.0,201.0,68.0,68.0,129.0,129.0,152.0,152.0,206.0,206.0
1,149.0,149.0,265.0,265.0,302.0,302.0,200.0,200.0,154.0,154.0,177.0,177.0,98.0,98.0,236.0,236.0,338.0,338.0,93.0,93.0,231.0,231.0,202.0,202.0,358.0,358.0,129.0,129.0,143.0,143.0,205.0,205.0
2,145.0,145.0,238.0,238.0,261.0,261.0,198.0,198.0,222.0,222.0,165.0,165.0,99.0,99.0,171.0,171.0,356.0,356.0,69.0,69.0,229.0,229.0,193.0,193.0,344.0,344.0,124.0,124.0,127.0,127.0,213.0,213.0
3,127.0,127.0,271.0,271.0,292.0,292.0,236.0,236.0,222.0,222.0,151.0,151.0,94.0,94.0,190.0,190.0,295.0,295.0,79.0,79.0,230.0,230.0,200.0,200.0,229.0,229.0,141.0,141.0,143.0,143.0,237.0,237.0
4,131.0,131.0,270.0,270.0,294.0,294.0,184.0,184.0,240.0,240.0,150.0,150.0,88.0,88.0,88.0,88.0,135.0,135.0,98.0,98.0,265.0,265.0,216.0,216.0,227.0,227.0,153.0,153.0,142.0,142.0,233.0,233.0


In [35]:
df_wind_dir.sample(10)

,KLO_Wind_dir,KLO_Wind_dir_delta,BIZ_Wind_dir,BIZ_Wind_dir_delta,BUS_Wind_dir,BUS_Wind_dir_delta,GUT_Wind_dir,GUT_Wind_dir_delta,KOP_Wind_dir,KOP_Wind_dir_delta,MOA_Wind_dir,MOA_Wind_dir_delta,MUB_Wind_dir,MUB_Wind_dir_delta,PAY_Wind_dir,PAY_Wind_dir_delta,PUY_Wind_dir,PUY_Wind_dir_delta,REH_Wind_dir,REH_Wind_dir_delta,SHA_Wind_dir,SHA_Wind_dir_delta,SMA_Wind_dir,SMA_Wind_dir_delta,TAE_Wind_dir,TAE_Wind_dir_delta,VAD_Wind_dir,VAD_Wind_dir_delta,WAE_Wind_dir,WAE_Wind_dir_delta,WYN_Wind_dir,WYN_Wind_dir_delta
34568,54.0,54.0,275.0,275.0,183.0,183.0,271.0,271.0,126.0,126.0,327.0,327.0,78.0,78.0,250.0,250.0,24.0,24.0,242.0,242.0,213.0,213.0,161.0,161.0,173.0,173.0,134.0,134.0,156.0,156.0,292.0,292.0
40729,309.0,309.0,226.0,226.0,178.0,178.0,308.0,308.0,99.0,99.0,295.0,295.0,0.0,0.0,216.0,216.0,15.0,15.0,291.0,291.0,31.0,31.0,39.0,39.0,175.0,175.0,44.0,44.0,261.0,261.0,212.0,212.0
223972,236.0,236.0,230.0,230.0,229.0,229.0,230.0,230.0,224.0,224.0,232.0,232.0,242.0,242.0,217.0,217.0,261.0,261.0,235.0,235.0,251.0,251.0,237.0,237.0,237.0,237.0,230.0,230.0,258.0,258.0,251.0,251.0
255790,335.0,335.0,312.0,312.0,216.0,216.0,315.0,315.0,107.0,107.0,313.0,313.0,86.0,86.0,206.0,206.0,46.0,46.0,300.0,300.0,56.0,56.0,245.0,245.0,231.0,231.0,163.0,163.0,301.0,301.0,221.0,221.0
74738,326.0,326.0,219.0,219.0,166.0,166.0,282.0,282.0,0.0,0.0,345.0,345.0,54.0,54.0,206.0,206.0,66.0,66.0,285.0,285.0,42.0,42.0,26.0,26.0,167.0,167.0,167.0,167.0,0.0,0.0,24.0,24.0
201198,214.0,214.0,239.0,239.0,247.0,247.0,232.0,232.0,202.0,202.0,128.0,128.0,281.0,281.0,211.0,211.0,335.0,335.0,243.0,243.0,238.0,238.0,224.0,224.0,222.0,222.0,165.0,165.0,332.0,332.0,241.0,241.0
92423,86.0,86.0,237.0,237.0,190.0,190.0,132.0,132.0,91.0,91.0,122.0,122.0,97.0,97.0,188.0,188.0,23.0,23.0,0.0,0.0,48.0,48.0,41.0,41.0,200.0,200.0,142.0,142.0,147.0,147.0,236.0,236.0
215048,207.0,207.0,225.0,225.0,272.0,272.0,230.0,230.0,222.0,222.0,130.0,130.0,146.0,146.0,198.0,198.0,112.0,112.0,301.0,301.0,199.0,199.0,226.0,226.0,216.0,216.0,141.0,141.0,244.0,244.0,211.0,211.0
125293,149.0,149.0,269.0,269.0,166.0,166.0,286.0,286.0,319.0,319.0,150.0,150.0,101.0,101.0,101.0,101.0,23.0,23.0,167.0,167.0,248.0,248.0,83.0,83.0,271.0,271.0,254.0,254.0,249.0,249.0,200.0,200.0
173898,330.0,330.0,317.0,317.0,327.0,327.0,328.0,328.0,321.0,321.0,355.0,355.0,313.0,313.0,6.0,6.0,232.0,232.0,305.0,305.0,343.0,343.0,320.0,320.0,342.0,342.0,332.0,332.0,325.0,325.0,322.0,322.0


In [0]:
for c in cols_wind_dir:
  
  df_wind_dir.loc[:,c + '_delta'] = df_wind_dir[c].diff()

In [44]:
df_wind_dir[25:40]

,KLO_Wind_dir,KLO_Wind_dir_delta,BIZ_Wind_dir,BIZ_Wind_dir_delta,BUS_Wind_dir,BUS_Wind_dir_delta,GUT_Wind_dir,GUT_Wind_dir_delta,KOP_Wind_dir,KOP_Wind_dir_delta,MOA_Wind_dir,MOA_Wind_dir_delta,MUB_Wind_dir,MUB_Wind_dir_delta,PAY_Wind_dir,PAY_Wind_dir_delta,PUY_Wind_dir,PUY_Wind_dir_delta,REH_Wind_dir,REH_Wind_dir_delta,SHA_Wind_dir,SHA_Wind_dir_delta,SMA_Wind_dir,SMA_Wind_dir_delta,TAE_Wind_dir,TAE_Wind_dir_delta,VAD_Wind_dir,VAD_Wind_dir_delta,WAE_Wind_dir,WAE_Wind_dir_delta,WYN_Wind_dir,WYN_Wind_dir_delta
25,139.0,17.0,56.0,4.0,308.0,23.0,154.0,-18.0,237.0,-13.0,160.0,6.0,116.0,3.0,174.0,-58.0,98.0,-19.0,113.0,39.0,255.0,5.0,167.0,-25.0,225.0,4.0,326.0,326.0,200.0,15.0,247.0,60.0
26,161.0,22.0,54.0,-2.0,328.0,20.0,159.0,5.0,226.0,-11.0,145.0,-15.0,112.0,-4.0,192.0,18.0,104.0,6.0,170.0,57.0,260.0,5.0,167.0,0.0,221.0,-4.0,19.0,-307.0,182.0,-18.0,216.0,-31.0
27,96.0,-65.0,77.0,23.0,323.0,-5.0,168.0,9.0,237.0,11.0,142.0,-3.0,118.0,6.0,218.0,26.0,93.0,-11.0,156.0,-14.0,252.0,-8.0,166.0,-1.0,227.0,6.0,176.0,157.0,128.0,-54.0,246.0,30.0
28,64.0,-32.0,66.0,-11.0,285.0,-38.0,149.0,-19.0,239.0,2.0,183.0,41.0,116.0,-2.0,225.0,7.0,76.0,-17.0,139.0,-17.0,257.0,5.0,178.0,12.0,224.0,-3.0,165.0,-11.0,154.0,26.0,267.0,21.0
29,42.0,-22.0,83.0,17.0,319.0,34.0,141.0,-8.0,236.0,-3.0,175.0,-8.0,110.0,-6.0,233.0,8.0,0.0,-76.0,110.0,-29.0,263.0,6.0,185.0,7.0,237.0,13.0,155.0,-10.0,153.0,-1.0,270.0,3.0
30,96.0,54.0,90.0,7.0,293.0,-26.0,177.0,36.0,250.0,14.0,138.0,-37.0,109.0,-1.0,329.0,96.0,3.0,3.0,143.0,33.0,268.0,5.0,208.0,23.0,239.0,2.0,155.0,0.0,107.0,-46.0,317.0,47.0
31,171.0,75.0,70.0,-20.0,13.0,-280.0,213.0,36.0,245.0,-5.0,123.0,-15.0,98.0,-11.0,0.0,-329.0,24.0,21.0,73.0,-70.0,263.0,-5.0,194.0,-14.0,224.0,-15.0,150.0,-5.0,123.0,16.0,0.0,-317.0
32,198.0,27.0,296.0,226.0,48.0,35.0,257.0,44.0,237.0,-8.0,199.0,76.0,106.0,8.0,193.0,193.0,0.0,-24.0,94.0,21.0,254.0,-9.0,209.0,15.0,219.0,-5.0,145.0,-5.0,114.0,-9.0,202.0,202.0
33,190.0,-8.0,232.0,-64.0,317.0,269.0,196.0,-61.0,223.0,-14.0,203.0,4.0,106.0,0.0,234.0,41.0,0.0,0.0,106.0,12.0,249.0,-5.0,193.0,-16.0,242.0,23.0,157.0,12.0,178.0,64.0,236.0,34.0
34,193.0,3.0,256.0,24.0,287.0,-30.0,184.0,-12.0,241.0,18.0,141.0,-62.0,106.0,0.0,311.0,77.0,0.0,0.0,73.0,-33.0,243.0,-6.0,196.0,3.0,269.0,27.0,135.0,-22.0,135.0,-43.0,225.0,-11.0


In [0]:
"""IF (((SUM([KLO Wind dir])-LOOKUP(SUM([KLO Wind dir]),-1)))>=(-180)
AND ((SUM([KLO Wind dir])-LOOKUP(SUM([KLO Wind dir]),-1))<=180))
THEN (SUM([KLO Wind dir])-LOOKUP(SUM([KLO Wind dir]),-1))
ELSEIF ((SUM([KLO Wind dir])-LOOKUP(SUM([KLO Wind dir]),-1))<(-180))
THEN (360+(SUM([KLO Wind dir])-LOOKUP(SUM([KLO Wind dir]),-1)))
ELSEIF((SUM([KLO Wind dir])-LOOKUP(SUM([KLO Wind dir]),-1))>180)
THEN ((SUM([KLO Wind dir])-LOOKUP(SUM([KLO Wind dir]),-1))-360)
END"""

def adjust_wind_dir_diff(x):
  if x < -180:
    return 360 + x
  elif x > 180:
    return x -360
  else:
    return x
    

In [69]:
cols_delta = [c for c in df_wind_dir.columns if re.search('_delta$', c)]

for c in cols_delta:
  print(c)
  df_wind_dir[c] = df_wind_dir[c].apply(lambda x: adjust_wind_dir_diff(x))

KLO_Wind_dir_delta
BIZ_Wind_dir_delta
BUS_Wind_dir_delta
GUT_Wind_dir_delta
KOP_Wind_dir_delta
MOA_Wind_dir_delta
MUB_Wind_dir_delta
PAY_Wind_dir_delta
PUY_Wind_dir_delta
REH_Wind_dir_delta
SHA_Wind_dir_delta
SMA_Wind_dir_delta
TAE_Wind_dir_delta
VAD_Wind_dir_delta
WAE_Wind_dir_delta
WYN_Wind_dir_delta


In [70]:
df_wind_dir[np.abs(df_wind_dir['KLO_Wind_dir_delta']) > 180][25:40] 

,KLO_Wind_dir,KLO_Wind_dir_delta,BIZ_Wind_dir,BIZ_Wind_dir_delta,BUS_Wind_dir,BUS_Wind_dir_delta,GUT_Wind_dir,GUT_Wind_dir_delta,KOP_Wind_dir,KOP_Wind_dir_delta,MOA_Wind_dir,MOA_Wind_dir_delta,MUB_Wind_dir,MUB_Wind_dir_delta,PAY_Wind_dir,PAY_Wind_dir_delta,PUY_Wind_dir,PUY_Wind_dir_delta,REH_Wind_dir,REH_Wind_dir_delta,SHA_Wind_dir,SHA_Wind_dir_delta,SMA_Wind_dir,SMA_Wind_dir_delta,TAE_Wind_dir,TAE_Wind_dir_delta,VAD_Wind_dir,VAD_Wind_dir_delta,WAE_Wind_dir,WAE_Wind_dir_delta,WYN_Wind_dir,WYN_Wind_dir_delta


In [71]:
df_wind_dir.head()

,KLO_Wind_dir,KLO_Wind_dir_delta,BIZ_Wind_dir,BIZ_Wind_dir_delta,BUS_Wind_dir,BUS_Wind_dir_delta,GUT_Wind_dir,GUT_Wind_dir_delta,KOP_Wind_dir,KOP_Wind_dir_delta,MOA_Wind_dir,MOA_Wind_dir_delta,MUB_Wind_dir,MUB_Wind_dir_delta,PAY_Wind_dir,PAY_Wind_dir_delta,PUY_Wind_dir,PUY_Wind_dir_delta,REH_Wind_dir,REH_Wind_dir_delta,SHA_Wind_dir,SHA_Wind_dir_delta,SMA_Wind_dir,SMA_Wind_dir_delta,TAE_Wind_dir,TAE_Wind_dir_delta,VAD_Wind_dir,VAD_Wind_dir_delta,WAE_Wind_dir,WAE_Wind_dir_delta,WYN_Wind_dir,WYN_Wind_dir_delta
0,138.0,NaN,283.0,NaN,303.0,NaN,207.0,NaN,191.0,NaN,146.0,NaN,94.0,NaN,262.0,NaN,9.0,NaN,104.0,NaN,233.0,NaN,201.0,NaN,68.0,NaN,129.0,NaN,152.0,NaN,206.0,NaN
1,149.0,11.0,265.0,-18.0,302.0,-1.0,200.0,-7.0,154.0,-37.0,177.0,31.0,98.0,4.0,236.0,-26.0,338.0,-31.0,93.0,-11.0,231.0,-2.0,202.0,1.0,358.0,-70.0,129.0,0.0,143.0,-9.0,205.0,-1.0
2,145.0,-4.0,238.0,-27.0,261.0,-41.0,198.0,-2.0,222.0,68.0,165.0,-12.0,99.0,1.0,171.0,-65.0,356.0,18.0,69.0,-24.0,229.0,-2.0,193.0,-9.0,344.0,-14.0,124.0,-5.0,127.0,-16.0,213.0,8.0
3,127.0,-18.0,271.0,33.0,292.0,31.0,236.0,38.0,222.0,0.0,151.0,-14.0,94.0,-5.0,190.0,19.0,295.0,-61.0,79.0,10.0,230.0,1.0,200.0,7.0,229.0,-115.0,141.0,17.0,143.0,16.0,237.0,24.0
4,131.0,4.0,270.0,-1.0,294.0,2.0,184.0,-52.0,240.0,18.0,150.0,-1.0,88.0,-6.0,88.0,-102.0,135.0,-160.0,98.0,19.0,265.0,35.0,216.0,16.0,227.0,-2.0,153.0,12.0,142.0,-1.0,233.0,-4.0


In [74]:
cols_delta_all = [c for c in df_master.columns if re.search('_dir_delta', c)]

for c in cols_delta_all:
  print('Correcting col', c)
  df_master[c] = df_master[c].apply(lambda x: adjust_wind_dir_diff(x))
  
df_master[np.abs(df_wind_dir['KLO_Wind_dir_delta']) > 180]

Correcting col KLO_Wind_dir_delta
Correcting col KLO_Wind_dir_delta-3
Correcting col KLO_Wind_dir_delta-6
Correcting col BIZ_Wind_dir_delta
Correcting col BIZ_Wind_dir_delta-3
Correcting col BIZ_Wind_dir_delta-6
Correcting col BUS_Wind_dir_delta
Correcting col BUS_Wind_dir_delta-3
Correcting col BUS_Wind_dir_delta-6
Correcting col GUT_Wind_dir_delta
Correcting col GUT_Wind_dir_delta-3
Correcting col GUT_Wind_dir_delta-6
Correcting col KOP_Wind_dir_delta
Correcting col KOP_Wind_dir_delta-3
Correcting col KOP_Wind_dir_delta-6
Correcting col MOA_Wind_dir_delta
Correcting col MOA_Wind_dir_delta-3
Correcting col MOA_Wind_dir_delta-6
Correcting col MUB_Wind_dir_delta
Correcting col MUB_Wind_dir_delta-3
Correcting col MUB_Wind_dir_delta-6
Correcting col PAY_Wind_dir_delta
Correcting col PAY_Wind_dir_delta-3
Correcting col PAY_Wind_dir_delta-6
Correcting col PUY_Wind_dir_delta
Correcting col PUY_Wind_dir_delta-3
Correcting col PUY_Wind_dir_delta-6
Correcting col REH_Wind_dir_delta
Correcting c

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys


,datetime,KLO_Air_temp,KLO_Hum,KLO_Hum_delta_3,KLO_Hum_delta_6,KLO_QNH,KLO_Wind_dir,KLO_Wind_dir_delta,KLO_Wind_dir_delta-3,KLO_Wind_dir_delta-6,KLO_Wind_dir_spread30,KLO_Wind_dir_spread60,KLO_Wind_max_EW,KLO_Wind_max_NS,KLO_Wind_max,KLO_Wind_max_delta_3,KLO_Wind_max_delta_6,KLO_Wind_speed_EW,KLO_Wind_speed_NS,KLO_Wind_speed,KLO_Wind_speed_delta_3,KLO_Wind_speed_delta_6,BIZ_Air_temp,BIZ_Hum,BIZ_Hum_delta_3,BIZ_Hum_delta_6,BIZ_QNH,BIZ_Wind_dir,BIZ_Wind_dir_delta,BIZ_Wind_dir_delta-3,BIZ_Wind_dir_delta-6,BIZ_Wind_dir_spread30,BIZ_Wind_dir_spread60,BIZ_Wind_max_EW,BIZ_Wind_max_NS,BIZ_Wind_max,BIZ_Wind_max_delta_3,BIZ_Wind_max_delta_6,BIZ_Wind_speed_EW,BIZ_Wind_speed_NS,BIZ_Wind_speed,BIZ_Wind_speed_delta_3,BIZ_Wind_speed_delta_6,BUS_Air_temp,BUS_Hum,BUS_Hum_delta_3,BUS_Hum_delta_6,BUS_QNH,BUS_Wind_dir,BUS_Wind_dir_delta,BUS_Wind_dir_delta-3,BUS_Wind_dir_delta-6,BUS_Wind_dir_spread30,BUS_Wind_dir_spread60,BUS_Wind_max_EW,BUS_Wind_max_NS,BUS_Wind_max,BUS_Wind_max_delta_3,BUS_Wind_max_delta_6,BUS_Wind_speed_EW,BUS_Wind_speed_NS,BUS_Wind_speed,BUS_Wind_speed_delta_3,BUS_Wind_speed_delta_6,GUT_Air_temp,GUT_Hum,GUT_Hum_delta_3,GUT_Hum_delta_6,GUT_QNH,GUT_Wind_dir,GUT_Wind_dir_delta,GUT_Wind_dir_delta-3,GUT_Wind_dir_delta-6,GUT_Wind_dir_spread30,GUT_Wind_dir_spread60,GUT_Wind_max_EW,GUT_Wind_max_NS,GUT_Wind_max,GUT_Wind_max_delta_3,GUT_Wind_max_delta_6,GUT_Wind_speed_EW,GUT_Wind_speed_NS,GUT_Wind_speed,GUT_Wind_speed_delta_3,GUT_Wind_speed_delta_6,KOP_Air_temp,KOP_Hum,KOP_Hum_delta_3,KOP_Hum_delta_6,KOP_QNH,KOP_Wind_dir,KOP_Wind_dir_delta,KOP_Wind_dir_delta-3,KOP_Wind_dir_delta-6,KOP_Wind_dir_spread30,KOP_Wind_dir_spread60,KOP_Wind_max_EW,KOP_Wind_max_NS,KOP_Wind_max,KOP_Wind_max_delta_3,KOP_Wind_max_delta_6,KOP_Wind_speed_EW,KOP_Wind_speed_NS,KOP_Wind_speed,KOP_Wind_speed_delta_3,KOP_Wind_speed_delta_6,MOA_Air_temp,MOA_Hum,MOA_Hum_delta_3,MOA_Hum_delta_6,MOA_QNH,MOA_Wind_dir,MOA_Wind_dir_delta,MOA_Wind_dir_delta-3,MOA_Wind_dir_delta-6,MOA_Wind_dir_spread30,MOA_Wind_dir_spread60,MOA_Wind_max_EW,MOA_Wind_max_NS,MOA_Wind_max,MOA_Wind_max_delta_3,MOA_Wind_max_delta_6,MOA_Wind_speed_EW,MOA_Wind_speed_NS,MOA_Wind_speed,MOA_Wind_speed_delta_3,MOA_Wind_speed_delta_6,MUB_Air_temp,MUB_Hum,MUB_Hum_delta_3,MUB_Hum_delta_6,MUB_QNH,MUB_Wind_dir,MUB_Wind_dir_delta,MUB_Wind_dir_delta-3,MUB_Wind_dir_delta-6,MUB_Wind_dir_spread30,MUB_Wind_dir_spread60,MUB_Wind_max_EW,MUB_Wind_max_NS,MUB_Wind_max,MUB_Wind_max_delta_3,MUB_Wind_max_delta_6,MUB_Wind_speed_EW,MUB_Wind_speed_NS,MUB_Wind_speed,MUB_Wind_speed_delta_3,MUB_Wind_speed_delta_6,PAY_Air_temp,PAY_Hum,PAY_Hum_delta_3,PAY_Hum_delta_6,PAY_QNH,PAY_Wind_dir,PAY_Wind_dir_delta,PAY_Wind_dir_delta-3,PAY_Wind_dir_delta-6,PAY_Wind_dir_spread30,PAY_Wind_dir_spread60,PAY_Wind_max_EW,PAY_Wind_max_NS,PAY_Wind_max,PAY_Wind_max_delta_3,PAY_Wind_max_delta_6,PAY_Wind_speed_EW,PAY_Wind_speed_NS,PAY_Wind_speed,PAY_Wind_speed_delta_3,PAY_Wind_speed_delta_6,PUY_Air_temp,PUY_Hum,PUY_Hum_delta_3,PUY_Hum_delta_6,PUY_QNH,PUY_Wind_dir,PUY_Wind_dir_delta,PUY_Wind_dir_delta-3,PUY_Wind_dir_delta-6,PUY_Wind_dir_spread30,PUY_Wind_dir_spread60,PUY_Wind_max_EW,PUY_Wind_max_NS,PUY_Wind_max,PUY_Wind_max_delta_3,PUY_Wind_max_delta_6,PUY_Wind_speed_EW,PUY_Wind_speed_NS,PUY_Wind_speed,PUY_Wind_speed_delta_3,PUY_Wind_speed_delta_6,REH_Air_temp,REH_Hum,REH_Hum_delta_3,REH_Hum_delta_6,REH_QNH,REH_Wind_dir,REH_Wind_dir_delta,REH_Wind_dir_delta-3,REH_Wind_dir_delta-6,REH_Wind_dir_spread30,REH_Wind_dir_spread60,REH_Wind_max_EW,REH_Wind_max_NS,REH_Wind_max,REH_Wind_max_delta_3,REH_Wind_max_delta_6,REH_Wind_speed_EW,REH_Wind_speed_NS,REH_Wind_speed,REH_Wind_speed_delta_3,REH_Wind_speed_delta_6,SHA_Air_temp,SHA_Hum,SHA_Hum_delta_3,SHA_Hum_delta_6,SHA_QNH,SHA_Wind_dir,SHA_Wind_dir_delta,SHA_Wind_dir_delta-3,SHA_Wind_dir_delta-6,SHA_Wind_dir_spread30,SHA_Wind_dir_spread60,SHA_Wind_max_EW,SHA_Wind_max_NS,SHA_Wind_max,SHA_Wind_max_delta_3,SHA_Wind_max_delta_6,SHA_Wind_speed_EW,SHA_Wind_speed_NS,SHA_Wind_speed,SHA_Wind_speed_delta_3,SHA_Wind_speed_delta

In [75]:
df_master.head()

,datetime,KLO_Air_temp,KLO_Hum,KLO_Hum_delta_3,KLO_Hum_delta_6,KLO_QNH,KLO_Wind_dir,KLO_Wind_dir_delta,KLO_Wind_dir_delta-3,KLO_Wind_dir_delta-6,KLO_Wind_dir_spread30,KLO_Wind_dir_spread60,KLO_Wind_max_EW,KLO_Wind_max_NS,KLO_Wind_max,KLO_Wind_max_delta_3,KLO_Wind_max_delta_6,KLO_Wind_speed_EW,KLO_Wind_speed_NS,KLO_Wind_speed,KLO_Wind_speed_delta_3,KLO_Wind_speed_delta_6,BIZ_Air_temp,BIZ_Hum,BIZ_Hum_delta_3,BIZ_Hum_delta_6,BIZ_QNH,BIZ_Wind_dir,BIZ_Wind_dir_delta,BIZ_Wind_dir_delta-3,BIZ_Wind_dir_delta-6,BIZ_Wind_dir_spread30,BIZ_Wind_dir_spread60,BIZ_Wind_max_EW,BIZ_Wind_max_NS,BIZ_Wind_max,BIZ_Wind_max_delta_3,BIZ_Wind_max_delta_6,BIZ_Wind_speed_EW,BIZ_Wind_speed_NS,BIZ_Wind_speed,BIZ_Wind_speed_delta_3,BIZ_Wind_speed_delta_6,BUS_Air_temp,BUS_Hum,BUS_Hum_delta_3,BUS_Hum_delta_6,BUS_QNH,BUS_Wind_dir,BUS_Wind_dir_delta,BUS_Wind_dir_delta-3,BUS_Wind_dir_delta-6,BUS_Wind_dir_spread30,BUS_Wind_dir_spread60,BUS_Wind_max_EW,BUS_Wind_max_NS,BUS_Wind_max,BUS_Wind_max_delta_3,BUS_Wind_max_delta_6,BUS_Wind_speed_EW,BUS_Wind_speed_NS,BUS_Wind_speed,BUS_Wind_speed_delta_3,BUS_Wind_speed_delta_6,GUT_Air_temp,GUT_Hum,GUT_Hum_delta_3,GUT_Hum_delta_6,GUT_QNH,GUT_Wind_dir,GUT_Wind_dir_delta,GUT_Wind_dir_delta-3,GUT_Wind_dir_delta-6,GUT_Wind_dir_spread30,GUT_Wind_dir_spread60,GUT_Wind_max_EW,GUT_Wind_max_NS,GUT_Wind_max,GUT_Wind_max_delta_3,GUT_Wind_max_delta_6,GUT_Wind_speed_EW,GUT_Wind_speed_NS,GUT_Wind_speed,GUT_Wind_speed_delta_3,GUT_Wind_speed_delta_6,KOP_Air_temp,KOP_Hum,KOP_Hum_delta_3,KOP_Hum_delta_6,KOP_QNH,KOP_Wind_dir,KOP_Wind_dir_delta,KOP_Wind_dir_delta-3,KOP_Wind_dir_delta-6,KOP_Wind_dir_spread30,KOP_Wind_dir_spread60,KOP_Wind_max_EW,KOP_Wind_max_NS,KOP_Wind_max,KOP_Wind_max_delta_3,KOP_Wind_max_delta_6,KOP_Wind_speed_EW,KOP_Wind_speed_NS,KOP_Wind_speed,KOP_Wind_speed_delta_3,KOP_Wind_speed_delta_6,MOA_Air_temp,MOA_Hum,MOA_Hum_delta_3,MOA_Hum_delta_6,MOA_QNH,MOA_Wind_dir,MOA_Wind_dir_delta,MOA_Wind_dir_delta-3,MOA_Wind_dir_delta-6,MOA_Wind_dir_spread30,MOA_Wind_dir_spread60,MOA_Wind_max_EW,MOA_Wind_max_NS,MOA_Wind_max,MOA_Wind_max_delta_3,MOA_Wind_max_delta_6,MOA_Wind_speed_EW,MOA_Wind_speed_NS,MOA_Wind_speed,MOA_Wind_speed_delta_3,MOA_Wind_speed_delta_6,MUB_Air_temp,MUB_Hum,MUB_Hum_delta_3,MUB_Hum_delta_6,MUB_QNH,MUB_Wind_dir,MUB_Wind_dir_delta,MUB_Wind_dir_delta-3,MUB_Wind_dir_delta-6,MUB_Wind_dir_spread30,MUB_Wind_dir_spread60,MUB_Wind_max_EW,MUB_Wind_max_NS,MUB_Wind_max,MUB_Wind_max_delta_3,MUB_Wind_max_delta_6,MUB_Wind_speed_EW,MUB_Wind_speed_NS,MUB_Wind_speed,MUB_Wind_speed_delta_3,MUB_Wind_speed_delta_6,PAY_Air_temp,PAY_Hum,PAY_Hum_delta_3,PAY_Hum_delta_6,PAY_QNH,PAY_Wind_dir,PAY_Wind_dir_delta,PAY_Wind_dir_delta-3,PAY_Wind_dir_delta-6,PAY_Wind_dir_spread30,PAY_Wind_dir_spread60,PAY_Wind_max_EW,PAY_Wind_max_NS,PAY_Wind_max,PAY_Wind_max_delta_3,PAY_Wind_max_delta_6,PAY_Wind_speed_EW,PAY_Wind_speed_NS,PAY_Wind_speed,PAY_Wind_speed_delta_3,PAY_Wind_speed_delta_6,PUY_Air_temp,PUY_Hum,PUY_Hum_delta_3,PUY_Hum_delta_6,PUY_QNH,PUY_Wind_dir,PUY_Wind_dir_delta,PUY_Wind_dir_delta-3,PUY_Wind_dir_delta-6,PUY_Wind_dir_spread30,PUY_Wind_dir_spread60,PUY_Wind_max_EW,PUY_Wind_max_NS,PUY_Wind_max,PUY_Wind_max_delta_3,PUY_Wind_max_delta_6,PUY_Wind_speed_EW,PUY_Wind_speed_NS,PUY_Wind_speed,PUY_Wind_speed_delta_3,PUY_Wind_speed_delta_6,REH_Air_temp,REH_Hum,REH_Hum_delta_3,REH_Hum_delta_6,REH_QNH,REH_Wind_dir,REH_Wind_dir_delta,REH_Wind_dir_delta-3,REH_Wind_dir_delta-6,REH_Wind_dir_spread30,REH_Wind_dir_spread60,REH_Wind_max_EW,REH_Wind_max_NS,REH_Wind_max,REH_Wind_max_delta_3,REH_Wind_max_delta_6,REH_Wind_speed_EW,REH_Wind_speed_NS,REH_Wind_speed,REH_Wind_speed_delta_3,REH_Wind_speed_delta_6,SHA_Air_temp,SHA_Hum,SHA_Hum_delta_3,SHA_Hum_delta_6,SHA_QNH,SHA_Wind_dir,SHA_Wind_dir_delta,SHA_Wind_dir_delta-3,SHA_Wind_dir_delta-6,SHA_Wind_dir_spread30,SHA_Wind_dir_spread60,SHA_Wind_max_EW,SHA_Wind_max_NS,SHA_Wind_max,SHA_Wind_max_delta_3,SHA_Wind_max_delta_6,SHA_Wind_speed_EW,SHA_Wind_speed_NS,SHA_Wind_speed,SHA_Wind_speed_delta_3,SHA_Wind_speed_delta

### Save the changes to file

In [0]:
 df_master.to_csv(file_master, compression='zip')

In [76]:
file_master

'gdrive/My Drive/bise_data/model_data_all_features_extended.csv.zip'

In [79]:
df_classified = pd.read_csv(join(data_path, 'model_data_all_features_classified.csv.zip'), index_col=0)
df_classified.head()

,datetime,original_index,year,month,bise_event,rule3_episode,rule4_episode,rule3_duration,rule4_duration,rule3_label,rule4_label,manual_bise_x,manual_bise_y,manual_duration_class,rule4_episode_1_0,rule4_duration_class,KLO_Air_temp,KLO_Hum,KLO_Hum_delta_3,KLO_Hum_delta_6,KLO_QNH,KLO_Wind_dir,KLO_Wind_dir_delta,KLO_Wind_dir_delta-3,KLO_Wind_dir_delta-6,KLO_Wind_dir_spread30,KLO_Wind_dir_spread60,KLO_Wind_max_EW,KLO_Wind_max_NS,KLO_Wind_max,KLO_Wind_max_delta_3,KLO_Wind_max_delta_6,KLO_Wind_speed_EW,KLO_Wind_speed_NS,KLO_Wind_speed,KLO_Wind_speed_delta_3,KLO_Wind_speed_delta_6,BIZ_Air_temp,BIZ_Hum,BIZ_Hum_delta_3,BIZ_Hum_delta_6,BIZ_QNH,BIZ_Wind_dir,BIZ_Wind_dir_delta,BIZ_Wind_dir_delta-3,BIZ_Wind_dir_delta-6,BIZ_Wind_dir_spread30,BIZ_Wind_dir_spread60,BIZ_Wind_max_EW,BIZ_Wind_max_NS,BIZ_Wind_max,BIZ_Wind_max_delta_3,BIZ_Wind_max_delta_6,BIZ_Wind_speed_EW,BIZ_Wind_speed_NS,BIZ_Wind_speed,BIZ_Wind_speed_delta_3,BIZ_Wind_speed_delta_6,BUS_Air_temp,BUS_Hum,BUS_Hum_delta_3,BUS_Hum_delta_6,BUS_QNH,BUS_Wind_dir,BUS_Wind_dir_delta,BUS_Wind_dir_delta-3,BUS_Wind_dir_delta-6,BUS_Wind_dir_spread30,BUS_Wind_dir_spread60,BUS_Wind_max_EW,BUS_Wind_max_NS,BUS_Wind_max,BUS_Wind_max_delta_3,BUS_Wind_max_delta_6,BUS_Wind_speed_EW,BUS_Wind_speed_NS,BUS_Wind_speed,BUS_Wind_speed_delta_3,BUS_Wind_speed_delta_6,GUT_Air_temp,GUT_Hum,GUT_Hum_delta_3,GUT_Hum_delta_6,GUT_QNH,GUT_Wind_dir,GUT_Wind_dir_delta,GUT_Wind_dir_delta-3,GUT_Wind_dir_delta-6,GUT_Wind_dir_spread30,GUT_Wind_dir_spread60,GUT_Wind_max_EW,GUT_Wind_max_NS,GUT_Wind_max,GUT_Wind_max_delta_3,GUT_Wind_max_delta_6,GUT_Wind_speed_EW,GUT_Wind_speed_NS,GUT_Wind_speed,GUT_Wind_speed_delta_3,GUT_Wind_speed_delta_6,KOP_Air_temp,KOP_Hum,KOP_Hum_delta_3,KOP_Hum_delta_6,KOP_QNH,KOP_Wind_dir,KOP_Wind_dir_delta,KOP_Wind_dir_delta-3,KOP_Wind_dir_delta-6,KOP_Wind_dir_spread30,KOP_Wind_dir_spread60,KOP_Wind_max_EW,KOP_Wind_max_NS,KOP_Wind_max,KOP_Wind_max_delta_3,KOP_Wind_max_delta_6,KOP_Wind_speed_EW,KOP_Wind_speed_NS,KOP_Wind_speed,KOP_Wind_speed_delta_3,KOP_Wind_speed_delta_6,MOA_Air_temp,MOA_Hum,MOA_Hum_delta_3,MOA_Hum_delta_6,MOA_QNH,MOA_Wind_dir,MOA_Wind_dir_delta,MOA_Wind_dir_delta-3,MOA_Wind_dir_delta-6,MOA_Wind_dir_spread30,MOA_Wind_dir_spread60,MOA_Wind_max_EW,MOA_Wind_max_NS,MOA_Wind_max,MOA_Wind_max_delta_3,MOA_Wind_max_delta_6,MOA_Wind_speed_EW,MOA_Wind_speed_NS,MOA_Wind_speed,MOA_Wind_speed_delta_3,MOA_Wind_speed_delta_6,MUB_Air_temp,MUB_Hum,MUB_Hum_delta_3,MUB_Hum_delta_6,MUB_QNH,MUB_Wind_dir,MUB_Wind_dir_delta,MUB_Wind_dir_delta-3,MUB_Wind_dir_delta-6,MUB_Wind_dir_spread30,MUB_Wind_dir_spread60,MUB_Wind_max_EW,MUB_Wind_max_NS,MUB_Wind_max,MUB_Wind_max_delta_3,MUB_Wind_max_delta_6,MUB_Wind_speed_EW,MUB_Wind_speed_NS,MUB_Wind_speed,MUB_Wind_speed_delta_3,MUB_Wind_speed_delta_6,PAY_Air_temp,PAY_Hum,PAY_Hum_delta_3,PAY_Hum_delta_6,PAY_QNH,PAY_Wind_dir,PAY_Wind_dir_delta,PAY_Wind_dir_delta-3,PAY_Wind_dir_delta-6,PAY_Wind_dir_spread30,PAY_Wind_dir_spread60,PAY_Wind_max_EW,PAY_Wind_max_NS,PAY_Wind_max,PAY_Wind_max_delta_3,PAY_Wind_max_delta_6,PAY_Wind_speed_EW,PAY_Wind_speed_NS,PAY_Wind_speed,PAY_Wind_speed_delta_3,PAY_Wind_speed_delta_6,PUY_Air_temp,PUY_Hum,PUY_Hum_delta_3,PUY_Hum_delta_6,PUY_QNH,PUY_Wind_dir,PUY_Wind_dir_delta,PUY_Wind_dir_delta-3,PUY_Wind_dir_delta-6,PUY_Wind_dir_spread30,PUY_Wind_dir_spread60,PUY_Wind_max_EW,PUY_Wind_max_NS,PUY_Wind_max,PUY_Wind_max_delta_3,PUY_Wind_max_delta_6,PUY_Wind_speed_EW,PUY_Wind_speed_NS,PUY_Wind_speed,PUY_Wind_speed_delta_3,PUY_Wind_speed_delta_6,REH_Air_temp,REH_Hum,REH_Hum_delta_3,REH_Hum_delta_6,REH_QNH,REH_Wind_dir,REH_Wind_dir_delta,REH_Wind_dir_delta-3,REH_Wind_dir_delta-6,REH_Wind_dir_spread30,REH_Wind_dir_spread60,REH_Wind_max_EW,REH_Wind_max_NS,REH_Wind_max,REH_Wind_max_delta_3,REH_Wind_max_delta_6,REH_Wind_speed_EW,REH_Wind_speed_NS,REH_Wind_speed,REH_Wind_speed_delta_3,REH_Wind_speed_delta_6,SHA_Air_temp,SHA_Hum,SHA_Hum_delta_3,SHA_Hum_delta_6,SHA_QNH,SHA_Wind_dir,SHA_Wind_dir_delta,SHA_Wind_dir_delta-3,SHA_Wind_dir_delta-6,SHA_Wind_dir_spre

In [81]:
df_classified.iloc[:,0:16].head()

,datetime,original_index,year,month,bise_event,rule3_episode,rule4_episode,rule3_duration,rule4_duration,rule3_label,rule4_label,manual_bise_x,manual_bise_y,manual_duration_class,rule4_episode_1_0,rule4_duration_class
0,2014-01-01 00:00:00,0,2014,1,0,0,0,0.0,0.0,0,0,0,0,0,0,0
1,2014-01-01 00:10:00,1,2014,1,0,0,0,0.0,0.0,0,0,0,0,0,0,0
2,2014-01-01 00:20:00,2,2014,1,0,0,0,0.0,0.0,0,0,0,0,0,0,0
3,2014-01-01 00:30:00,3,2014,1,0,0,0,0.0,0.0,0,0,0,0,0,0,0
4,2014-01-01 00:40:00,4,2014,1,0,0,0,0.0,0.0,0,0,0,0,0,0,0


In [82]:
df_class_new = dp.left_join_datasets(df_classified.iloc[:,0:16],df_master)
df_class_new.head()

,datetime,original_index,year,month,bise_event,rule3_episode,rule4_episode,rule3_duration,rule4_duration,rule3_label,rule4_label,manual_bise_x,manual_bise_y,manual_duration_class,rule4_episode_1_0,rule4_duration_class,KLO_Air_temp,KLO_Hum,KLO_Hum_delta_3,KLO_Hum_delta_6,KLO_QNH,KLO_Wind_dir,KLO_Wind_dir_delta,KLO_Wind_dir_delta-3,KLO_Wind_dir_delta-6,KLO_Wind_dir_spread30,KLO_Wind_dir_spread60,KLO_Wind_max_EW,KLO_Wind_max_NS,KLO_Wind_max,KLO_Wind_max_delta_3,KLO_Wind_max_delta_6,KLO_Wind_speed_EW,KLO_Wind_speed_NS,KLO_Wind_speed,KLO_Wind_speed_delta_3,KLO_Wind_speed_delta_6,BIZ_Air_temp,BIZ_Hum,BIZ_Hum_delta_3,BIZ_Hum_delta_6,BIZ_QNH,BIZ_Wind_dir,BIZ_Wind_dir_delta,BIZ_Wind_dir_delta-3,BIZ_Wind_dir_delta-6,BIZ_Wind_dir_spread30,BIZ_Wind_dir_spread60,BIZ_Wind_max_EW,BIZ_Wind_max_NS,BIZ_Wind_max,BIZ_Wind_max_delta_3,BIZ_Wind_max_delta_6,BIZ_Wind_speed_EW,BIZ_Wind_speed_NS,BIZ_Wind_speed,BIZ_Wind_speed_delta_3,BIZ_Wind_speed_delta_6,BUS_Air_temp,BUS_Hum,BUS_Hum_delta_3,BUS_Hum_delta_6,BUS_QNH,BUS_Wind_dir,BUS_Wind_dir_delta,BUS_Wind_dir_delta-3,BUS_Wind_dir_delta-6,BUS_Wind_dir_spread30,BUS_Wind_dir_spread60,BUS_Wind_max_EW,BUS_Wind_max_NS,BUS_Wind_max,BUS_Wind_max_delta_3,BUS_Wind_max_delta_6,BUS_Wind_speed_EW,BUS_Wind_speed_NS,BUS_Wind_speed,BUS_Wind_speed_delta_3,BUS_Wind_speed_delta_6,GUT_Air_temp,GUT_Hum,GUT_Hum_delta_3,GUT_Hum_delta_6,GUT_QNH,GUT_Wind_dir,GUT_Wind_dir_delta,GUT_Wind_dir_delta-3,GUT_Wind_dir_delta-6,GUT_Wind_dir_spread30,GUT_Wind_dir_spread60,GUT_Wind_max_EW,GUT_Wind_max_NS,GUT_Wind_max,GUT_Wind_max_delta_3,GUT_Wind_max_delta_6,GUT_Wind_speed_EW,GUT_Wind_speed_NS,GUT_Wind_speed,GUT_Wind_speed_delta_3,GUT_Wind_speed_delta_6,KOP_Air_temp,KOP_Hum,KOP_Hum_delta_3,KOP_Hum_delta_6,KOP_QNH,KOP_Wind_dir,KOP_Wind_dir_delta,KOP_Wind_dir_delta-3,KOP_Wind_dir_delta-6,KOP_Wind_dir_spread30,KOP_Wind_dir_spread60,KOP_Wind_max_EW,KOP_Wind_max_NS,KOP_Wind_max,KOP_Wind_max_delta_3,KOP_Wind_max_delta_6,KOP_Wind_speed_EW,KOP_Wind_speed_NS,KOP_Wind_speed,KOP_Wind_speed_delta_3,KOP_Wind_speed_delta_6,MOA_Air_temp,MOA_Hum,MOA_Hum_delta_3,MOA_Hum_delta_6,MOA_QNH,MOA_Wind_dir,MOA_Wind_dir_delta,MOA_Wind_dir_delta-3,MOA_Wind_dir_delta-6,MOA_Wind_dir_spread30,MOA_Wind_dir_spread60,MOA_Wind_max_EW,MOA_Wind_max_NS,MOA_Wind_max,MOA_Wind_max_delta_3,MOA_Wind_max_delta_6,MOA_Wind_speed_EW,MOA_Wind_speed_NS,MOA_Wind_speed,MOA_Wind_speed_delta_3,MOA_Wind_speed_delta_6,MUB_Air_temp,MUB_Hum,MUB_Hum_delta_3,MUB_Hum_delta_6,MUB_QNH,MUB_Wind_dir,MUB_Wind_dir_delta,MUB_Wind_dir_delta-3,MUB_Wind_dir_delta-6,MUB_Wind_dir_spread30,MUB_Wind_dir_spread60,MUB_Wind_max_EW,MUB_Wind_max_NS,MUB_Wind_max,MUB_Wind_max_delta_3,MUB_Wind_max_delta_6,MUB_Wind_speed_EW,MUB_Wind_speed_NS,MUB_Wind_speed,MUB_Wind_speed_delta_3,MUB_Wind_speed_delta_6,PAY_Air_temp,PAY_Hum,PAY_Hum_delta_3,PAY_Hum_delta_6,PAY_QNH,PAY_Wind_dir,PAY_Wind_dir_delta,PAY_Wind_dir_delta-3,PAY_Wind_dir_delta-6,PAY_Wind_dir_spread30,PAY_Wind_dir_spread60,PAY_Wind_max_EW,PAY_Wind_max_NS,PAY_Wind_max,PAY_Wind_max_delta_3,PAY_Wind_max_delta_6,PAY_Wind_speed_EW,PAY_Wind_speed_NS,PAY_Wind_speed,PAY_Wind_speed_delta_3,PAY_Wind_speed_delta_6,PUY_Air_temp,PUY_Hum,PUY_Hum_delta_3,PUY_Hum_delta_6,PUY_QNH,PUY_Wind_dir,PUY_Wind_dir_delta,PUY_Wind_dir_delta-3,PUY_Wind_dir_delta-6,PUY_Wind_dir_spread30,PUY_Wind_dir_spread60,PUY_Wind_max_EW,PUY_Wind_max_NS,PUY_Wind_max,PUY_Wind_max_delta_3,PUY_Wind_max_delta_6,PUY_Wind_speed_EW,PUY_Wind_speed_NS,PUY_Wind_speed,PUY_Wind_speed_delta_3,PUY_Wind_speed_delta_6,REH_Air_temp,REH_Hum,REH_Hum_delta_3,REH_Hum_delta_6,REH_QNH,REH_Wind_dir,REH_Wind_dir_delta,REH_Wind_dir_delta-3,REH_Wind_dir_delta-6,REH_Wind_dir_spread30,REH_Wind_dir_spread60,REH_Wind_max_EW,REH_Wind_max_NS,REH_Wind_max,REH_Wind_max_delta_3,REH_Wind_max_delta_6,REH_Wind_speed_EW,REH_Wind_speed_NS,REH_Wind_speed,REH_Wind_speed_delta_3,REH_Wind_speed_delta_6,SHA_Air_temp,SHA_Hum,SHA_Hum_delta_3,SHA_Hum_delta_6,SHA_QNH,SHA_Wind_dir,SHA_Wind_dir_delta,SHA_Wind_dir_delta-3,SHA_Wind_dir_delta-6,SHA_Wind_dir_spre

In [0]:
df_class_new.to_csv(join(data_path, 'model_data_all_features_classified.csv.zip'), compression='zip')


### Notebook crashed while saving 'model_data_all_features_classified.csv.zip' in pevious cell, file probably corrupted

In [7]:
file_master = join(data_path, dh.master_dataset())
df_master = pd.read_csv(file_master, index_col=0)
df_master.head()

,datetime,KLO_Air_temp,KLO_Hum,KLO_Hum_delta_3,KLO_Hum_delta_6,KLO_QNH,KLO_Wind_dir,KLO_Wind_dir_delta,KLO_Wind_dir_delta-3,KLO_Wind_dir_delta-6,KLO_Wind_dir_spread30,KLO_Wind_dir_spread60,KLO_Wind_max_EW,KLO_Wind_max_NS,KLO_Wind_max,KLO_Wind_max_delta_3,KLO_Wind_max_delta_6,KLO_Wind_speed_EW,KLO_Wind_speed_NS,KLO_Wind_speed,KLO_Wind_speed_delta_3,KLO_Wind_speed_delta_6,BIZ_Air_temp,BIZ_Hum,BIZ_Hum_delta_3,BIZ_Hum_delta_6,BIZ_QNH,BIZ_Wind_dir,BIZ_Wind_dir_delta,BIZ_Wind_dir_delta-3,BIZ_Wind_dir_delta-6,BIZ_Wind_dir_spread30,BIZ_Wind_dir_spread60,BIZ_Wind_max_EW,BIZ_Wind_max_NS,BIZ_Wind_max,BIZ_Wind_max_delta_3,BIZ_Wind_max_delta_6,BIZ_Wind_speed_EW,BIZ_Wind_speed_NS,BIZ_Wind_speed,BIZ_Wind_speed_delta_3,BIZ_Wind_speed_delta_6,BUS_Air_temp,BUS_Hum,BUS_Hum_delta_3,BUS_Hum_delta_6,BUS_QNH,BUS_Wind_dir,BUS_Wind_dir_delta,BUS_Wind_dir_delta-3,BUS_Wind_dir_delta-6,BUS_Wind_dir_spread30,BUS_Wind_dir_spread60,BUS_Wind_max_EW,BUS_Wind_max_NS,BUS_Wind_max,BUS_Wind_max_delta_3,BUS_Wind_max_delta_6,BUS_Wind_speed_EW,BUS_Wind_speed_NS,BUS_Wind_speed,BUS_Wind_speed_delta_3,BUS_Wind_speed_delta_6,GUT_Air_temp,GUT_Hum,GUT_Hum_delta_3,GUT_Hum_delta_6,GUT_QNH,GUT_Wind_dir,GUT_Wind_dir_delta,GUT_Wind_dir_delta-3,GUT_Wind_dir_delta-6,GUT_Wind_dir_spread30,GUT_Wind_dir_spread60,GUT_Wind_max_EW,GUT_Wind_max_NS,GUT_Wind_max,GUT_Wind_max_delta_3,GUT_Wind_max_delta_6,GUT_Wind_speed_EW,GUT_Wind_speed_NS,GUT_Wind_speed,GUT_Wind_speed_delta_3,GUT_Wind_speed_delta_6,KOP_Air_temp,KOP_Hum,KOP_Hum_delta_3,KOP_Hum_delta_6,KOP_QNH,KOP_Wind_dir,KOP_Wind_dir_delta,KOP_Wind_dir_delta-3,KOP_Wind_dir_delta-6,KOP_Wind_dir_spread30,KOP_Wind_dir_spread60,KOP_Wind_max_EW,KOP_Wind_max_NS,KOP_Wind_max,KOP_Wind_max_delta_3,KOP_Wind_max_delta_6,KOP_Wind_speed_EW,KOP_Wind_speed_NS,KOP_Wind_speed,KOP_Wind_speed_delta_3,KOP_Wind_speed_delta_6,MOA_Air_temp,MOA_Hum,MOA_Hum_delta_3,MOA_Hum_delta_6,MOA_QNH,MOA_Wind_dir,MOA_Wind_dir_delta,MOA_Wind_dir_delta-3,MOA_Wind_dir_delta-6,MOA_Wind_dir_spread30,MOA_Wind_dir_spread60,MOA_Wind_max_EW,MOA_Wind_max_NS,MOA_Wind_max,MOA_Wind_max_delta_3,MOA_Wind_max_delta_6,MOA_Wind_speed_EW,MOA_Wind_speed_NS,MOA_Wind_speed,MOA_Wind_speed_delta_3,MOA_Wind_speed_delta_6,MUB_Air_temp,MUB_Hum,MUB_Hum_delta_3,MUB_Hum_delta_6,MUB_QNH,MUB_Wind_dir,MUB_Wind_dir_delta,MUB_Wind_dir_delta-3,MUB_Wind_dir_delta-6,MUB_Wind_dir_spread30,MUB_Wind_dir_spread60,MUB_Wind_max_EW,MUB_Wind_max_NS,MUB_Wind_max,MUB_Wind_max_delta_3,MUB_Wind_max_delta_6,MUB_Wind_speed_EW,MUB_Wind_speed_NS,MUB_Wind_speed,MUB_Wind_speed_delta_3,MUB_Wind_speed_delta_6,PAY_Air_temp,PAY_Hum,PAY_Hum_delta_3,PAY_Hum_delta_6,PAY_QNH,PAY_Wind_dir,PAY_Wind_dir_delta,PAY_Wind_dir_delta-3,PAY_Wind_dir_delta-6,PAY_Wind_dir_spread30,PAY_Wind_dir_spread60,PAY_Wind_max_EW,PAY_Wind_max_NS,PAY_Wind_max,PAY_Wind_max_delta_3,PAY_Wind_max_delta_6,PAY_Wind_speed_EW,PAY_Wind_speed_NS,PAY_Wind_speed,PAY_Wind_speed_delta_3,PAY_Wind_speed_delta_6,PUY_Air_temp,PUY_Hum,PUY_Hum_delta_3,PUY_Hum_delta_6,PUY_QNH,PUY_Wind_dir,PUY_Wind_dir_delta,PUY_Wind_dir_delta-3,PUY_Wind_dir_delta-6,PUY_Wind_dir_spread30,PUY_Wind_dir_spread60,PUY_Wind_max_EW,PUY_Wind_max_NS,PUY_Wind_max,PUY_Wind_max_delta_3,PUY_Wind_max_delta_6,PUY_Wind_speed_EW,PUY_Wind_speed_NS,PUY_Wind_speed,PUY_Wind_speed_delta_3,PUY_Wind_speed_delta_6,REH_Air_temp,REH_Hum,REH_Hum_delta_3,REH_Hum_delta_6,REH_QNH,REH_Wind_dir,REH_Wind_dir_delta,REH_Wind_dir_delta-3,REH_Wind_dir_delta-6,REH_Wind_dir_spread30,REH_Wind_dir_spread60,REH_Wind_max_EW,REH_Wind_max_NS,REH_Wind_max,REH_Wind_max_delta_3,REH_Wind_max_delta_6,REH_Wind_speed_EW,REH_Wind_speed_NS,REH_Wind_speed,REH_Wind_speed_delta_3,REH_Wind_speed_delta_6,SHA_Air_temp,SHA_Hum,SHA_Hum_delta_3,SHA_Hum_delta_6,SHA_QNH,SHA_Wind_dir,SHA_Wind_dir_delta,SHA_Wind_dir_delta-3,SHA_Wind_dir_delta-6,SHA_Wind_dir_spread30,SHA_Wind_dir_spread60,SHA_Wind_max_EW,SHA_Wind_max_NS,SHA_Wind_max,SHA_Wind_max_delta_3,SHA_Wind_max_delta_6,SHA_Wind_speed_EW,SHA_Wind_speed_NS,SHA_Wind_speed,SHA_Wind_speed_delta_3,SHA_Wind_speed_delta

In [9]:
df_master[np.abs(df_master['KLO_Wind_dir_delta']) > 180]

,datetime,KLO_Air_temp,KLO_Hum,KLO_Hum_delta_3,KLO_Hum_delta_6,KLO_QNH,KLO_Wind_dir,KLO_Wind_dir_delta,KLO_Wind_dir_delta-3,KLO_Wind_dir_delta-6,KLO_Wind_dir_spread30,KLO_Wind_dir_spread60,KLO_Wind_max_EW,KLO_Wind_max_NS,KLO_Wind_max,KLO_Wind_max_delta_3,KLO_Wind_max_delta_6,KLO_Wind_speed_EW,KLO_Wind_speed_NS,KLO_Wind_speed,KLO_Wind_speed_delta_3,KLO_Wind_speed_delta_6,BIZ_Air_temp,BIZ_Hum,BIZ_Hum_delta_3,BIZ_Hum_delta_6,BIZ_QNH,BIZ_Wind_dir,BIZ_Wind_dir_delta,BIZ_Wind_dir_delta-3,BIZ_Wind_dir_delta-6,BIZ_Wind_dir_spread30,BIZ_Wind_dir_spread60,BIZ_Wind_max_EW,BIZ_Wind_max_NS,BIZ_Wind_max,BIZ_Wind_max_delta_3,BIZ_Wind_max_delta_6,BIZ_Wind_speed_EW,BIZ_Wind_speed_NS,BIZ_Wind_speed,BIZ_Wind_speed_delta_3,BIZ_Wind_speed_delta_6,BUS_Air_temp,BUS_Hum,BUS_Hum_delta_3,BUS_Hum_delta_6,BUS_QNH,BUS_Wind_dir,BUS_Wind_dir_delta,BUS_Wind_dir_delta-3,BUS_Wind_dir_delta-6,BUS_Wind_dir_spread30,BUS_Wind_dir_spread60,BUS_Wind_max_EW,BUS_Wind_max_NS,BUS_Wind_max,BUS_Wind_max_delta_3,BUS_Wind_max_delta_6,BUS_Wind_speed_EW,BUS_Wind_speed_NS,BUS_Wind_speed,BUS_Wind_speed_delta_3,BUS_Wind_speed_delta_6,GUT_Air_temp,GUT_Hum,GUT_Hum_delta_3,GUT_Hum_delta_6,GUT_QNH,GUT_Wind_dir,GUT_Wind_dir_delta,GUT_Wind_dir_delta-3,GUT_Wind_dir_delta-6,GUT_Wind_dir_spread30,GUT_Wind_dir_spread60,GUT_Wind_max_EW,GUT_Wind_max_NS,GUT_Wind_max,GUT_Wind_max_delta_3,GUT_Wind_max_delta_6,GUT_Wind_speed_EW,GUT_Wind_speed_NS,GUT_Wind_speed,GUT_Wind_speed_delta_3,GUT_Wind_speed_delta_6,KOP_Air_temp,KOP_Hum,KOP_Hum_delta_3,KOP_Hum_delta_6,KOP_QNH,KOP_Wind_dir,KOP_Wind_dir_delta,KOP_Wind_dir_delta-3,KOP_Wind_dir_delta-6,KOP_Wind_dir_spread30,KOP_Wind_dir_spread60,KOP_Wind_max_EW,KOP_Wind_max_NS,KOP_Wind_max,KOP_Wind_max_delta_3,KOP_Wind_max_delta_6,KOP_Wind_speed_EW,KOP_Wind_speed_NS,KOP_Wind_speed,KOP_Wind_speed_delta_3,KOP_Wind_speed_delta_6,MOA_Air_temp,MOA_Hum,MOA_Hum_delta_3,MOA_Hum_delta_6,MOA_QNH,MOA_Wind_dir,MOA_Wind_dir_delta,MOA_Wind_dir_delta-3,MOA_Wind_dir_delta-6,MOA_Wind_dir_spread30,MOA_Wind_dir_spread60,MOA_Wind_max_EW,MOA_Wind_max_NS,MOA_Wind_max,MOA_Wind_max_delta_3,MOA_Wind_max_delta_6,MOA_Wind_speed_EW,MOA_Wind_speed_NS,MOA_Wind_speed,MOA_Wind_speed_delta_3,MOA_Wind_speed_delta_6,MUB_Air_temp,MUB_Hum,MUB_Hum_delta_3,MUB_Hum_delta_6,MUB_QNH,MUB_Wind_dir,MUB_Wind_dir_delta,MUB_Wind_dir_delta-3,MUB_Wind_dir_delta-6,MUB_Wind_dir_spread30,MUB_Wind_dir_spread60,MUB_Wind_max_EW,MUB_Wind_max_NS,MUB_Wind_max,MUB_Wind_max_delta_3,MUB_Wind_max_delta_6,MUB_Wind_speed_EW,MUB_Wind_speed_NS,MUB_Wind_speed,MUB_Wind_speed_delta_3,MUB_Wind_speed_delta_6,PAY_Air_temp,PAY_Hum,PAY_Hum_delta_3,PAY_Hum_delta_6,PAY_QNH,PAY_Wind_dir,PAY_Wind_dir_delta,PAY_Wind_dir_delta-3,PAY_Wind_dir_delta-6,PAY_Wind_dir_spread30,PAY_Wind_dir_spread60,PAY_Wind_max_EW,PAY_Wind_max_NS,PAY_Wind_max,PAY_Wind_max_delta_3,PAY_Wind_max_delta_6,PAY_Wind_speed_EW,PAY_Wind_speed_NS,PAY_Wind_speed,PAY_Wind_speed_delta_3,PAY_Wind_speed_delta_6,PUY_Air_temp,PUY_Hum,PUY_Hum_delta_3,PUY_Hum_delta_6,PUY_QNH,PUY_Wind_dir,PUY_Wind_dir_delta,PUY_Wind_dir_delta-3,PUY_Wind_dir_delta-6,PUY_Wind_dir_spread30,PUY_Wind_dir_spread60,PUY_Wind_max_EW,PUY_Wind_max_NS,PUY_Wind_max,PUY_Wind_max_delta_3,PUY_Wind_max_delta_6,PUY_Wind_speed_EW,PUY_Wind_speed_NS,PUY_Wind_speed,PUY_Wind_speed_delta_3,PUY_Wind_speed_delta_6,REH_Air_temp,REH_Hum,REH_Hum_delta_3,REH_Hum_delta_6,REH_QNH,REH_Wind_dir,REH_Wind_dir_delta,REH_Wind_dir_delta-3,REH_Wind_dir_delta-6,REH_Wind_dir_spread30,REH_Wind_dir_spread60,REH_Wind_max_EW,REH_Wind_max_NS,REH_Wind_max,REH_Wind_max_delta_3,REH_Wind_max_delta_6,REH_Wind_speed_EW,REH_Wind_speed_NS,REH_Wind_speed,REH_Wind_speed_delta_3,REH_Wind_speed_delta_6,SHA_Air_temp,SHA_Hum,SHA_Hum_delta_3,SHA_Hum_delta_6,SHA_QNH,SHA_Wind_dir,SHA_Wind_dir_delta,SHA_Wind_dir_delta-3,SHA_Wind_dir_delta-6,SHA_Wind_dir_spread30,SHA_Wind_dir_spread60,SHA_Wind_max_EW,SHA_Wind_max_NS,SHA_Wind_max,SHA_Wind_max_delta_3,SHA_Wind_max_delta_6,SHA_Wind_speed_EW,SHA_Wind_speed_NS,SHA_Wind_speed,SHA_Wind_speed_delta_3,SHA_Wind_speed_delta

### Uncompressed for download, zip files get corrupted on their way down from Google Drive

In [0]:
df_master.to_csv(join(data_path, 'model_data_all_features_extended.csv'))

In [10]:
dh.master_dataset()

'model_data_all_features_extended.csv.zip'

In [14]:
df_classified = pd.read_csv(join(data_path, 'model_data_all_features_classified.csv.zip'), index_col=0)
df_classified.head()

BadZipFile: ignored